<div style="text-align: center; padding-top: 30px; padding-bottom: 10px;">

<h1 style="font-size: 2.8em; font-weight: 600; margin-bottom: 0.2em;">
Monte Carlo simulations
</h1>

<p style="font-size: 1.2em; color: gray; font-style: italic; margin-top: 0;">
This notebook shows the results from the simulation study
</p>

</div>


## 1. Loading Packages and data


In [2]:
from simulations.simulation_functions import simulate, Surface
import numpy as np
import plotly.graph_objects as go
from utils import create_pred_input

2025-09-03 06:53:33.030172: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-03 06:53:33.618326: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-03 06:53:40.439393: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-03 06:53:45.539244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756882429.629741     882 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756882430.76

In [3]:

def calculate_surface(specification, element, date, model):
    data= simulate(seed=0, n_countries=196, n_years=63, specification=specification, add_noise=True, sample_data=True, dynamic=False)


    _, T, P = create_pred_input(
        True,
        mean_T=np.mean(data["temperature"]),
        std_T=np.std(data["temperature"]),
        mean_P=np.mean(data["precipitation"]),
        std_P=np.std(data["precipitation"])
    )

    surface=np.load(f"/workspaces/Paper_1/results/MonteCarlo/{specification}/{model}/{date}/surfaces.np.npy")
    # # # surface=np.load(f"/workspaces/Paper_1/outputs/2025-08-14/12-56-44/results/MonteCarlo/q_Leirvik/2025-08-14/_avg_surface.np.npy")
    model_values = np.mean(surface, axis=0).reshape(T.shape)
    benchmark = Surface(T.flatten(), P.flatten(), specification=specification, dynamic=False, time_periods=63).reshape(T.shape)

    
    if element == 'model':
    
        surf = go.Surface(
            x=T, y=P, z=model_values,
            colorscale='Plasma',
            opacity=0.85,
            showscale=False,
            name=f'Model'
        )

    elif element == 'benchmark':
        surf= go.Surface(
            x=T, y=P, z=benchmark,
            colorscale='Plasma',
            opacity=0.85,
            showscale=False,
            name=f'Benchmark'
        )
    
    else: 
        bias = np.array(model_values - benchmark)

        zmin, zmax = 0, 1
        ticks = np.linspace(zmin, zmax, 9)           # 9 tick positions
        ticktext = [f"{t:.1f}" for t in ticks]       # optional: nicely formatted labels

        surf = go.Heatmap(
            x=P[:, 0], y=T[0, :], z=np.sqrt(np.square(bias.T)),
            colorscale='ylorrd', zmin=zmin, zmax=zmax,
            showscale=True,
            colorbar=dict(
                title='Bias',
                thickness=15,
                len=1,
                tickmode='array',
                tickvals=ticks,
                ticktext=ticktext,
            )
        )
    fig=create_figure(element, surf)
    
  
    if element=="bias":
        return fig, bias
    else:
        return fig


def create_figure(element, plot_data):
    
    if element=="model" or element=="benchmark":
        fig = go.Figure(data=plot_data)
        fig.update_layout(
               autosize=False,
        width=500,
        height=600,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4,
        ),

            scene=dict(
                xaxis_title='Temperature (°C)',
                yaxis_title='Precipitation (m)',
                zaxis_title='Δ ln(Growth)',
                camera=dict(eye=dict(x=2.35, y=0.006, z=0.4))
                
            ),
            
            legend=dict(
                bgcolor='rgba(255,255,255,0.7)',
                bordercolor='black',
                borderwidth=1
            ),
            font=dict(
            size=10
        )
        )
        
    else: 
        layout = go.Layout(
        yaxis=dict(autorange='reversed'),
        font=dict(
            size=16
        ))
        fig = go.Figure(data=plot_data, layout=layout)
        fig.update_layout(
        xaxis=dict(title='Precipitation (m)'),
        yaxis=dict(title='Temperature (°C)')
    )



    return fig


In [16]:


fig, bias =calculate_surface("Trig", "bias", "2025-08-28", "Interactive")
#data, plot, date, specification
fig.show()
average_bias=np.mean(np.sqrt(bias**2))
print(f'Average bias: {average_bias}')




Average bias: 0.19623621829896729


In [59]:
#optionally save as pdf 
fig.write_image("../results/images/Monte Carlo/bias_Trig_Interactive.pdf")

In [15]:
surface=np.load(f"/workspaces/Paper_1/results/MonteCarlo/linear/NN/2025-08-25/surfaces.np.npy")
    # surface=np.load(f"/workspaces/Paper_1/outputs/2025-08-14/12-56-44/results/MonteCarlo/q_Leirvik/2025-08-14/_avg_surface.np.npy")
print(surface)

[[ 1.2044461   1.1861778   1.1676731  ...  0.1575681   0.16068856
   0.16384682]
 [-1.5353681  -1.5108845  -1.4862542  ...  0.6787489   0.67797023
   0.6773794 ]]
